# Pruebas sobre el dataset dev

## Primera parte. Entrenar modelo sobre dataset completo

In [1]:
import pandas as pd
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..', 'src')))


In [2]:
# Cargar el conjunto de entrenamiento
df = pd.read_csv('../data/TA1C_dataset_detection_train.csv')
df = df.drop(['Tweet Date', 'Media Name', 'Media Origin'], axis=1)

In [ ]:
from preprocessing import normalize_text

df["cleaned_lemmatized_text"] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="text_cleaning")).apply(lambda x: normalize_text(x, mode="lemmatization"))

df.to_csv('../data_dev/dataset_normalizado.csv', index=False)

In [4]:
df_normalized = pd.read_csv('../data_dev/dataset_normalizado')

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Probar diferentes configuraciones de n-gramas y representaciones
configurations = [
    {"ngram_range": (1, 1), "representation": "tfidf"},  # Unigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "tfidf"},  # Unigramas + Bigramas con TF-IDF
    {"ngram_range": (1, 3), "representation": "tfidf"},  # Unigramas + Bigramas + Trigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "binary"},  # Unigramas + Bigramas con representación binaria
    {"ngram_range": (1, 2), "representation": "frequency"},  # Unigramas + Bigramas con frecuencia
]

for config in configurations:
    print(f"Configuración: ngram_range={config['ngram_range']}, representación={config['representation']}")
    
    # Configurar el vectorizador
    if config["representation"] == "tfidf":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"])
    elif config["representation"] == "binary":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], binary=True)
    elif config["representation"] == "frequency":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], use_idf=False)
    
    # Crear representaciones para el conjunto de entrenamiento y desarrollo
    X = vectorizer.fit_transform(df_normalized['cleaned_lemmatized_text'])
    
    # Verificar las dimensiones
    print(f"Dimensiones de X: {X.shape}")

label_mapping = {'Clickbait': 1, 'No': 0}
df['Tag Value'] = df['Tag Value'].map(label_mapping)
y = df['Tag Value']

Configuración: ngram_range=(1, 1), representación=tfidf
Dimensiones de X: (2800, 10591)
Configuración: ngram_range=(1, 2), representación=tfidf
Dimensiones de X: (2800, 47819)
Configuración: ngram_range=(1, 3), representación=tfidf
Dimensiones de X: (2800, 98988)
Configuración: ngram_range=(1, 2), representación=binary
Dimensiones de X: (2800, 47819)
Configuración: ngram_range=(1, 2), representación=frequency
Dimensiones de X: (2800, 47819)


In [6]:
from imblearn.combine import SMOTETomek

smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X, y)

In [9]:
from sklearn.decomposition import TruncatedSVD

# Reducir dimensionalidad con TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=0)  # Reducir a 100 dimensiones
X_reduced = svd.fit_transform(X_resampled)

# Verificar las dimensiones después de la reducción
print(f"Dimensiones reducidas de X_reduced: {X_reduced.shape}")

Dimensiones reducidas de X_reduced: (4004, 100)


In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer

# Definir el modelo y los hiperparámetros a probar
mlp = MLPClassifier(random_state=0, max_iter=2000)
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (150,)],
    'alpha': [0.0001, 0.001, 0.01],
    'solver': ['adam', 'lbfgs'],
}

# Configurar validación cruzada y búsqueda de hiperparámetros
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
grid_search = GridSearchCV(
    mlp,
    param_grid,
    scoring=make_scorer(f1_score, average='macro'),
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Ejecutar búsqueda
grid_search.fit(X_reduced, y_resampled)

# Imprimir mejores resultados
print("Mejor configuración encontrada:")
print(grid_search.best_params_)
print(f"Mejor F1-macro: {grid_search.best_score_:.4f}")

print("\nResultados detallados por configuración:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f"{param} -> F1-macro: {mean:.4f} (+/-{std:.4f})")

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  13.4s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  15.1s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  17.5s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  19.9s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   7.7s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   6.9s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  12.6s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   6.0s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   9.3s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=  10.0s
[CV] END alpha=0.0001, hidden_layer_sizes=(100,), solver=adam; total time=  17.8s
[CV] END alpha=0.0001, hidden_layer_sizes=(100,), solver=adam; total time=  13.2s
[CV] END alpha=0.0001

/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END .alpha=0.01, hidden_layer_sizes=(50,), solver=lbfgs; total time=  21.8s


/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

[CV] END .alpha=0.01, hidden_layer_sizes=(50,), solver=lbfgs; total time=  20.3s
[CV] END .alpha=0.01, hidden_layer_sizes=(100,), solver=adam; total time=  15.5s
[CV] END .alpha=0.01, hidden_layer_sizes=(100,), solver=adam; total time=  13.9s
[CV] END .alpha=0.01, hidden_layer_sizes=(100,), solver=adam; total time=  18.6s
[CV] END alpha=0.01, hidden_layer_sizes=(100,), solver=lbfgs; total time=  31.5s
[CV] END alpha=0.01, hidden_layer_sizes=(100,), solver=lbfgs; total time=  31.7s
[CV] END alpha=0.01, hidden_layer_sizes=(100,), solver=lbfgs; total time=  40.5s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=adam; total time=  17.9s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=adam; total time=  16.3s


/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END alpha=0.01, hidden_layer_sizes=(100,), solver=lbfgs; total time=  45.9s
[CV] END alpha=0.01, hidden_layer_sizes=(100,), solver=lbfgs; total time=  37.1s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=adam; total time=  16.9s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=adam; total time=  14.0s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=adam; total time=  13.8s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=lbfgs; total time=  43.1s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=lbfgs; total time=  47.5s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=lbfgs; total time=  40.6s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=lbfgs; total time=  47.2s
[CV] END .alpha=0.01, hidden_layer_sizes=(150,), solver=adam; total time=  17.6s
[CV] END .alpha=0.01, hidden_layer_sizes=(150,), solver=adam; total time=  18.5s
[CV] END .alpha=0.01, hidden_layer_sizes=(150,), solver=adam; total time=  13.9s
[CV] END .

/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END alpha=0.01, hidden_layer_sizes=(150,), solver=lbfgs; total time=  37.2s
Mejor configuración encontrada:
{'alpha': 0.001, 'hidden_layer_sizes': (150,), 'solver': 'adam'}
Mejor F1-macro: 0.8851

Resultados detallados por configuración:
{'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'solver': 'adam'} -> F1-macro: 0.8759 (+/-0.0096)
{'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'solver': 'lbfgs'} -> F1-macro: 0.8483 (+/-0.0060)
{'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'solver': 'adam'} -> F1-macro: 0.8851 (+/-0.0103)
{'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'solver': 'lbfgs'} -> F1-macro: 0.8443 (+/-0.0146)
{'alpha': 0.0001, 'hidden_layer_sizes': (100, 50), 'solver': 'adam'} -> F1-macro: 0.8735 (+/-0.0105)
{'alpha': 0.0001, 'hidden_layer_sizes': (100, 50), 'solver': 'lbfgs'} -> F1-macro: 0.8466 (+/-0.0093)
{'alpha': 0.0001, 'hidden_layer_sizes': (150,), 'solver': 'adam'} -> F1-macro: 0.8811 (+/-0.0091)
{'alpha': 0.0001, 'hidden_layer_sizes': (150,), 'solver': 'lbfgs

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

# Definir el modelo de MLP
mlp_model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=2000, alpha=0.001, solver='adam', random_state=0)

# Evaluar el modelo con validación cruzada usando las representaciones reducidas (X_train_reduced)
mlp_scores = cross_val_score(mlp_model, X_reduced, y_resampled, cv=5, scoring='f1_macro')

# Imprimir los resultados
print("Multi-layer Perceptron (MLPClassifier):")
print(f"F1-macro scores por fold: {mlp_scores}")
print(f"F1-macro promedio: {mlp_scores.mean():.4f}")

mlp_model.fit(X_reduced, y_resampled)

Multi-layer Perceptron (MLPClassifier):
F1-macro scores por fold: [0.8243289  0.83185766 0.93874054 0.92869492 0.92852384]
F1-macro promedio: 0.8904


MLPClassifier(alpha=0.001, hidden_layer_sizes=(50,), max_iter=2000,
              random_state=0)

In [17]:
import joblib

joblib.dump(mlp_model, '../model/mlp_model.joblib')
joblib.dump(vectorizer, '../model/vectorizer.joblib')
joblib.dump(svd, '../model/svd.joblib')

['../model/svd.joblib']